## Data Manipulation with the ORM

The previous section `Working with Data` remained focused on the _SQL Expression Language_ from a _Core perspective_, in order to __provide continuity__ across the _major SQL statement constructs_. This section will then build out the lifecycle of the `Session` and __how it interacts with these constructs__.

**Prerequisite Sections** - the ORM focused part builds upon two previous _ORM-centric sections_ in this document:

* __Executing with an ORM Session__ - introduces how to make an ORM `Session` object.

* __Defining Table Metadata with the ORM__ - where we set up our _ORM mappings_ of the `User` and `Address` entities.

* __Selecting ORM Entities and Columns__ - a few examples on _how to run SELECT statements_ for entities like `User`.

#### Inserting Rows with the ORM

When using the ORM, the `Session` object is responsible for __constructing Insert constructs__ and __emitting__ them for us in a _transaction_. The way we instruct the `Session` to do so is by __adding object entries__ to it; the `Session` then makes sure these _new entries_ will be _emitted to the database_ when they are needed, using a process known as a __`flush`__.

In [1]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import Session, registry, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [3]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [4]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [5]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

##### Instances of Classes represent Rows

Whereas in the previous example we __emitted an `INSERT`__ using Python dictionaries to indicate the _data we wanted to add_, with the ORM we make _direct use of the custom Python classes_ we defined, back at `Defining Table Metadata with the ORM`. At the _class level_, the `User` and `Address` classes served as a place to define what the __corresponding database tables__ should look like. These classes also serve as __extensible data objects__ that we use to _create and manipulate rows within a transaction_ as well. Below we will create two `User` objects each representing a _potential database row_ to be `INSERT`ed.

In [6]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

We are able to construct these objects using the _names of the mapped columns as keyword arguments_ in the constructor. This is possible as the `User` class includes an __automatically generated__ `__init__()` constructor that was _provided by the ORM mapping_ so that we could create each object using _column names as keys in the constructor_.

In a similar manner as in our _Core examples_ of `Insert`, we __did not include a primary key__ (i.e. an entry for the `id` column), since we would like to make use of the _auto-incrementing primary key feature of the database_, `SQLite` in this case, which the ORM also integrates with. The value of the `id` attribute on the above objects, if we were to view it, displays itself as `None`.

In [7]:
print(f"{squidward=}")

squidward=User(id=None, name='squidward', fullname='Squidward Tentacles')


The `None` value is provided by `SQLAlchemy` to indicate that the __attribute has no value__ as of yet. _SQLAlchemy-mapped attributes_ always __return a value in Python__ and _don't raise AttributeError if they're missing_, when dealing with a new object that has not had a value assigned.

At the moment, our two objects above are said to be in a _state_ called `transient` - they are __`not associated` with any database state__ and are __yet to be associated with a `Session` object__ that can _generate_ `INSERT` statements for them.

##### Adding objects to a Session

To illustrate the _addition_ process step by step, we will create a `Session` without using a _context manager_ (and hence we must make sure we close it later!).

In [8]:
session = Session(engine)

The __objects are then added to the `Session`__ using the `Session.add()` method. When this is called, the objects are in a _state_ known as __`pending`__ and _have not been inserted_ yet.

In [9]:
session.add(squidward)
session.add(krabs)

When we have _pending objects_, we can see this _state_ by looking at a _collection_ on the `Session` called `Session.new`.

In [10]:
session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

The above view is using a __collection__ called `IdentitySet` that is _essentially_ a `Python set` that __hashes on object identity__ in all cases (i.e., using Python built-in `id()` function, rather than the Python `hash()` function).